Import Drive to Colab

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Import libraries

In [2]:
import numpy as np
import os
import re
import matplotlib.pyplot as plt
%matplotlib inline
import skimage
import tensorflow as tf
import keras

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.utils import to_categorical
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import ELU
from tensorflow.keras.applications.vgg16 import VGG16
from skimage.transform import resize

Reading images for training

In [ ]:
dirname = os.path.join('/content/drive/MyDrive/', 'ParticionADNI1 (training)')
imgpath = dirname + os.sep

imagestrain1 = []
directories = []
dircount = []
prevRoot=''
cant=0

print("leyendo imagenes de ",imgpath)

for root, dirnames, filenames in os.walk(imgpath):
  dirnames.sort()
  for filename in filenames:
      if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
          cant=cant+1
          filepath = os.path.join(root, filename)
          image = plt.imread(filepath)
          if (image.shape!=(256,256)):
            image_resized = resize(image, (256, 256), anti_aliasing=True)
            imagestrain1.append(image_resized)
          else:
            imagestrain1.append(image)
          b = "Leyendo..." + str(cant)
          print (b, end="\r")
          if prevRoot !=root:
              print(root, cant)
              prevRoot=root
              directories.append(root)
              dircount.append(cant)
              cant=0
dircount.append(cant)

dircount = dircount[1:]
dircount[0]=dircount[0]+1

print('Directorios leidos:',len(directories))
print("Imagenes en cada directorio", dircount)
print('suma Total de imagenes en subdirs:',sum(dircount))
print('Directorios leidos:',len(directories))
print("Imagenes en cada directorio", dircount)
print('suma Total de imagenes en subdirs:',sum(dircount))

labels=[]
indice=0
for cantidad in dircount:
    for i in range(cantidad):
        labels.append(indice)
    indice=indice+1
print("Cantidad etiquetas creadas: ",len(labels))

oasis=[]
indice=0
for directorio in directories:
    name = directorio.split(os.sep)
    print(indice , name[len(name)-1])
    oasis.append(name[len(name)-1])
    indice=indice+1

Reading images for testing

In [ ]:
dirname = os.path.join('/content/drive/MyDrive/', 'ParticionADNI1 (test)')
imgpath = dirname + os.sep

imagestest1 = []
directories1 = []
dircount1 = []
prevRoot=''
cant=0

print("leyendo imagenes de ",imgpath)

for root, dirnames, filenames in os.walk(imgpath):
  dirnames.sort()
  for filename in filenames:
      if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
          cant=cant+1
          filepath = os.path.join(root, filename)
          image = plt.imread(filepath)
          if (image.shape!=(256,256)):
            image_resized = resize(image, (256, 256), anti_aliasing=True)
            imagestest1.append(image_resized)
          else:
            imagestest1.append(image)
          b = "Leyendo..." + str(cant)
          print (b, end="\r")
          if prevRoot !=root:
              print(root, cant)
              prevRoot=root
              directories1.append(root)
              dircount1.append(cant)
              cant=0
dircount1.append(cant)

dircount1 = dircount1[1:]
dircount1[0] = dircount1[0]+1

print('Directorios leidos:',len(directories1))
print("Imagenes en cada directorio", dircount1)
print('suma Total de imagenes en subdirs:',sum(dircount1))
print('Directorios leidos:',len(directories1))
print("Imagenes en cada directorio", dircount1)
print('suma Total de imagenes en subdirs:',sum(dircount1))

labels1=[]
indice=0
for cantidad in dircount1:
    for i in range(cantidad):
        labels1.append(indice)
    indice=indice+1
print("Cantidad etiquetas creadas: ",len(labels1))

oasis=[]
indice=0
for directorio in directories1:
    name = directorio.split(os.sep)
    print(indice , name[len(name)-1])
    oasis.append(name[len(name)-1])
    indice=indice+1

Data preprocessing

In [12]:
y = np.array(labels)
X = np.array(imagestrain1, dtype=np.uint8) #convierto de lista a numpy

x2 = []

for i in X:
    x2.append(skimage.color.gray2rgb(i))
x2 = np.array(x2)

X = x2

tf.keras.applications.vgg16.preprocess_input(
    X, data_format=None
)

y_test = np.array(labels1)
x_test = np.array(imagestest1, dtype=np.uint8) #convierto de lista a numpy

x3 = []

for i in x_test:
    x3.append(skimage.color.gray2rgb(i))
x3 = np.array(x3)

x_test = x3


In [ ]:
train_X = X
test_X = x_test
train_Y = y
test_Y = y_test
print('Training data shape : ', train_X.shape, train_Y.shape)
print('Testing data shape : ', test_X.shape, test_Y.shape)

train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X = train_X / 255.
test_X = test_X / 255.

# Change the labels from categorical to one-hot encoding
train_Y_one_hot = to_categorical(train_Y)
test_Y_one_hot = to_categorical(test_Y)

# Display the change for category label using one-hot encoding
print('Original label:', train_Y[0])
print('After conversion to one-hot:', train_Y_one_hot[0])

train_label = train_Y_one_hot
valid_X = test_X
valid_label = test_Y_one_hot

Building the architecture and training the model

In [ ]:
INIT_LR = 1e-3
epochs = 100
batch_size = 16

conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(256,256,3))
conv_base.summary()
for layer in conv_base.layers:
    layer.trainable = False
conv_base.summary()

oasis_model = Sequential()
oasis_model.add(conv_base)
oasis_model.add(Flatten())
oasis_model.add(Dense(32, activation='linear'))
oasis_model.add(ELU(alpha=1))
oasis_model.add(Dropout(0.5))
oasis_model.add(Dense(2, activation='softmax'))
oasis_model.summary()

oasis_model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.legacy.Adagrad(learning_rate=INIT_LR, decay=INIT_LR / 100),metrics=['accuracy', 'MeanSquaredError', 'AUC', 'Precision', 'Recall'])
oasis_train = oasis_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))
